## Python wrapper for SWMM
Test case for Gottesacker model by Zhao Chen.
***
**Goals:**
<br>- Writes SWMM project.inp files using a template (template.inp) - searches for placeholder strings in the template and replaces them with data from a pandas dataframe. Dataframes can be created manually, or loaded from .csv files.
<br>- Run SWMM using the specified project.inp file
<br>- Import results from project.rpt file as pandas dataframes
***
**Files needed:**
<br>- template.inp
<br>- swmm5.exe
<br>- swmm5.dll
<br>
**Packages needed:**
<br>- pandas
<br>- numpy
<br>- subprocess

In [163]:
import pandas as pd
import numpy as np
import subprocess

In [158]:
#File structure and naming:

template_filename = 'template.inp'                                          #filename for template .inp file to import
input_filename =    'project.inp'                                              #filename to write new .inp file to
data_filenames =    ['junctions.csv', 'conduits.csv']                       #csv filenames with data to insert into the .inp file
placeholders =      ['junctions', 'conduits']                               #placeholder strings to search for and replace with data


In [159]:
def import_template(template_filename):
    '''Imports the template.inp file into a pandas dataframe.
    Inputs:
    template_filename: filename/path for the template.inp file
                       this must be a text file correctly formatted for SWMM, with placeholder strings in the sections to be edited
                       to create a sample template file, use the tutorial1.inp file from the SWMM documentation, and replace the data
                       in the [JUNCTIONS] section with the word 'junctions'
    Outputs:
    template:          pandas dataframe of the template file, with one column, and a line for each line of text.'''
    
    template =   pd.read_csv(template_filename, header=None, skip_blank_lines=False) #import template .inp file to dataframe
    return template
    

In [160]:
def import_data(data_filename):
    '''Imports data from a csv file into a pandas dataframe with columns.
    Inputs:
    data_filename:     filename/path for the data.csv file for a section
                       this must be a csv file of data formatted with the first row as a comment row, 
                       and the second row with the correct SWMM column headers for that section 
                       (see SWMM documentation for description of data in each section)
    Outputs:
    template:          pandas dataframe of the data file, with columns, and a line for each item'''
    
    data =  pd.read_csv(data_filename,     header=1)                            #import data to dataframe that has separate columns
    return data

In [161]:
def insert_data(template, placeholder, data, show=False):
    '''Inserts data from a pandas dataframe into a template dataframe by replacing the section placeholder in the template.
    The template file and data file must be read in first. Best to make a copy of the template.
    
    Inputs:
    template:     pandas dataframe with one column, and one line per line of text in the template input file.
                  NOTE: each section that will be edited MUST have a section placeholder string at the desired insert location, 
                  and a section header string with dashes for the column spacing, like so: ;;---- ------ ------
    placeholder:  string corresponding to the placeholder string to be replaced
    data:         pandas dataframe with rows of data split into columns, to be inserted at the placeholder location
                  NOTE: the first column will get converted to an integer
    show:         True/False. If True, prints the updated section
    
    Outputs:
    new:          pandas dataframe with one column, and one line per line of text in the desired output file, 
                  with the new data inserted where the placeholder was in the original file
    '''
    
    #Get row index and location of placeholder:
    ind = template.index[template[0]==placeholder]                  #df.index gets the index, df[0] looks in column 0 (returns an index object not just the name)
    loc = template.index.get_loc(ind[0])                        #get the integer position of the index specified (need to select first item (index name) in index object)

    #Convert data to formatted line strings:
    #Create format string based on template file:
    fmt_line = template[0].loc[loc-1]                           #get string to model formatting on (the row of dashes in the template file)
    cols = fmt_line.split()                                     #split string into chunks separated by whitespace
    l = [len(item) for item in cols]                            #get length of each column
    form = ''                                                   #create empty string to fill
    form = [form + '{d['+ str(i) + ']:<' + str(l[i]+1) + '}' for i in range(len(l))]  #concatenate formatting info and return a list of format strings (one per column)
    form = ''.join(form)                                        #join format strings into one for entire row

    #Insert values into format string
    data = data.round(6)                                #round data to 6 decimal places (to correct for artifacts caused by binary/float conversions)
    data_strings = pd.DataFrame(columns=[0])            #create empty dataframe to fill
    for ind in data.index:                              #loop over lines of data
        dl = data.loc[ind].tolist()                     #get line of data to be formatted
        dl[0] = int(dl[0])                              #make sure name column is an integer not a float (WHY is this not automatic?)
        line = form.format(d=dl)                        #insert each item in list into format string
        data_strings.loc[ind] = line                    #insert line string into new dataframe

    #Replace placeholder with new data strings:
    #Split original df into two, one for everything before the placeholder, one for everything after:
    dfA = template[template.index < loc]                            #create df for everything above placeholder
    dfB = template[template.index > loc]                            #create df for everything below placeholder

    #Append the three dfs to each other (part above, part to insert, part below):
    new = dfA.append(data_strings, ignore_index=True)               #append additional part to top part
    new = new.append(dfB,          ignore_index=True)               #append bottom part to new df
    
    if show==True:
        print(new.iloc[loc-2:loc+len(data_strings)])               #print updated section if desired

    return new

In [162]:
#Insert data into desired sections:

template = import_template(template_filename)                               #import template file from txt
for i in range(len(placeholders)):                                          #loop over list of placeholders
    data = import_data(data_filenames[i])                                   #import data to insert from csv
    template = insert_data(template,  placeholders[i], data)                #replace placeholder string with data
    template.to_csv(input_filename, header=False, index=False)              #write dataframe to .inp text file with specified name

In [191]:
#Run SWMM with new .inp and generate .rpt and .out files

#Define program & file names to use:
exe = 'swmm5.exe'
inputfile = 'project.inp'
reportfile = 'project.rpt'
outputfile = 'project.out'

#Run SWMM and print process:
p = subprocess.Popen([exe, inputfile, reportfile, outputfile], stdout=subprocess.PIPE, universal_newlines=True)          #run SWMM (and report process output)
for line in p.stdout:          #for each line of process output
    if 'hour' not in line:     #if the line doesn't include the string 'hour' (to avoid a huge mass of text for each timestep)
        print(line)            



... EPA-SWMM 5.0 (Build 5.0.022)



 o  Retrieving project data



... EPA-SWMM completed in 48.00 seconds.



In [196]:
#Read report file:

report = pd.read_table('project.rpt', header=None)  #read report file into dataframe
print(report)

                                                     0
0      EPA STORM WATER MANAGEMENT MODEL - VERSION 5...
1      --------------------------------------------...
2      Template input file for Gottesacker karst sy...
3          Based on input files created by Zhao Chen. 
4                           Modified by Chloe Fandel: 
5      ********************************************...
6      NOTE: The summary statistics displayed in th...
7      based on results found at every computationa...
8      not just on results from each reporting time...
9      ********************************************...
10                                    ****************
11                                    Analysis Options
12                                    ****************
13                      Flow Units ............... LPS
14                                     Process Models:
15                        Rainfall/Runoff ........ YES
16                         Snowmelt ............... NO
17        

In [189]:
#Problem with floats in data:

data = pd.read_csv('conduits.csv', header=1)
data = data.round(6)
dl = data.loc[37]
print(dl)
n = data.loc[37].ManningN
print(n)


ds = dl.tolist()    #this is where the problem is - adds extra digits to ManningN column - but why?
print(ds)


Name             2039
InNode           1036
OutNode          1037
Length         547.12
ManningN     0.029422
InOffset            *
OutOffset           *
InitFlow            0
MaxFlow             0
Name: 37, dtype: object
0.029422
[2039, 1036, 1037, 547.12, 0.029422, '*', '*', 0, 0]
